# Email Parsing

First, we need to import the required libraries.

In [1]:
import imaplib
import email
from email.parser import HeaderParser
import re
from email.header import decode_header
import csv

Get the all email data form Spam folder.

In [2]:
pwfile = open("account.txt","r")
lines=pwfile.readlines()
un = lines[0]
pw = lines[1]
conn = imaplib.IMAP4_SSL(port = '993',host = 'imap.gmail.com')
conn.login(un,pw)
conn.select('[Gmail]/Spam')
type, data = conn.search(None, 'ALL')
emaillist=data[0].split()

Get email headers

In [3]:
parser = HeaderParser()
header_list=[]
for a in emaillist:
    type, data2 = conn.fetch(a, '(RFC822)')
    h = parser.parsestr(data2[0][1].decode('utf-8','ignore'))
    header = {}
    header['Subject']=decode_header(h['Subject'])[0][0]
    header['Sender-ip']= str(re.findall(r'(?:(?:25[0-5]|2[0-4]\d|[01]?[0-9][0-9]?)\.){3}(?:25[0-5]|2[0-4]\d|[01]?\d\d?)',str(header)))
    header['ARC-Authentication-Results']=h['ARC-Authentication-Results'].strip()
    header['Return-Path']=h['Return-Path'].strip()
    header['Received']=h['Received'].strip()
    header['Received-SPF']=h['Received-SPF'].strip()
    header['Date']=h['Date'].strip()
    if 'Reply-To' in h:
        header['Reply-To']=h['Reply-To'].strip()
    else:
        header['Reply-To']=''    
    header['Content-Type']=h['Content-Type'].strip()
    header_list.append(header)

In [4]:
keys = header_list[0].keys()
with open('spam.csv', 'w') as output_file:
    dict_writer = csv.DictWriter(output_file, keys)
    dict_writer.writeheader()
    dict_writer.writerows(header_list)